In [1]:
import numpy as np
from ondelette import *
import tensorflow as tf
import tensorflow_io as tfio

dtype = tf.float32
rng = np.random.default_rng()

2025-03-09 13:02:54.036685: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 13:02:54.040514: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-09 13:02:54.053154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-09 13:02:54.078899: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-09 13:02:54.078948: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 13:02:54.094593: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
def stretch(length, tensor):
    old_length = tensor.shape[1]
    factor = old_length / length
    indexes = tf.range(0, length, dtype=tf.float32)
    scaled = indexes * factor
    int_indexes = tf.cast(scaled, dtype=tf.int32)
    return tf.gather(tensor, int_indexes, axis=1)

def stretch_dec(base_length, levels):
    lengths = [base_length]
    lengths += [base_length * (2 ** i) for i in range(len(levels) - 1)]
    return [stretch(l, t) for l, t in zip(lengths, levels)]


In [5]:
song = 'MusicDelta_Rock'

audio = tfio.audio.AudioIOTensor(f'data/MedleyDB_V1/{song}_MIX.mp3', dtype=tf.float32)
rate = tf.cast(audio.rate, dtype=tf.int64)
raw_samples = audio.to_tensor()
samples = tf.transpose(raw_samples)
print(f'samples: {samples.shape[1] // 44100} sec')

for name, make in wavelets.all.items():
    print(f'{name}...')

    wavelet = make(dtype=dtype)
    filters = wavelets.dwt_filters(wavelet)
    inverse_filters = wavelets.idwt_filters(wavelet)

    levels = wavelets.wavedec(filters, 11, samples)
    frames = framing.levels_to_frames(levels)
    length = frames[0].shape[0]

    stretched = framing.stretch(length * 2, frames)
    unframed = framing.frames_to_levels(stretched)

    rec = wavelets.waverec(inverse_filters, unframed)
    print(f'   reconstruction: {rec.shape[1] // 44100} sec')
    channels = tf.transpose(rec)
    mp3 = tfio.audio.encode_mp3(channels, rate)
    tf.io.write_file(f'temp/{name}.mp3', mp3)


samples: 13 sec
bior1.1...
   reconstruction: 26 sec
bior1.3...
   reconstruction: 26 sec
bior1.5...
   reconstruction: 26 sec
bior2.2...
   reconstruction: 26 sec
bior2.4...
   reconstruction: 26 sec
bior2.6...
   reconstruction: 26 sec
bior2.8...
   reconstruction: 26 sec
bior3.1...
   reconstruction: 26 sec
bior3.3...
   reconstruction: 26 sec
bior3.5...
   reconstruction: 26 sec
bior3.7...
   reconstruction: 26 sec
bior3.9...
   reconstruction: 27 sec
bior4.4...
   reconstruction: 26 sec
bior5.5...
   reconstruction: 26 sec
bior6.8...
   reconstruction: 26 sec
coif1...
   reconstruction: 26 sec
coif2...
   reconstruction: 26 sec
coif3...
   reconstruction: 26 sec
coif4...
   reconstruction: 27 sec
coif5...
   reconstruction: 27 sec
coif6...
   reconstruction: 27 sec
coif7...
   reconstruction: 28 sec
coif8...
   reconstruction: 28 sec
coif9...
   reconstruction: 28 sec
coif10...
   reconstruction: 28 sec
coif11...
   reconstruction: 29 sec
coif12...
   reconstruction: 29 sec
coif13